# Machine Learning Engineer Nanodegree

## Capstone Project

### Predicting the mortality of cancer patients in ICESP


In [24]:
from os.path import isfile, isdir
import urllib
import pandas as pd
from zipfile import ZipFile
# full dataset https://github.com/vitorrmachado/udacity_mlnanodegree_capstone_proposal/raw/master/dataset.zip
# 10000 samples https://github.com/vitorrmachado/udacity_mlnanodegree_capstone_proposal/raw/master/dataset-samples.zip
dataset_file = 'datasets/dataset-samples.csv'
if not isfile(dataset_file):
    dataset_zip = 'datasets/dataset.zip'
    url = 'https://github.com/vitorrmachado/udacity_mlnanodegree_capstone_proposal/raw/master/dataset-samples.zip'
    print "downloading dataset"
    urllib.urlretrieve(url, dataset_zip)
    zip = ZipFile(dataset_zip)
    zip.extractall('datasets')
data = pd.read_csv(dataset_file)
print 'Dataset loaded!'

downloading dataset


ParserError: Error tokenizing data. C error: Expected 3 fields in line 28, saw 4
